In [6]:
#Bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args

#Variaveis para armazenamento da semente utilizada para geração de números randomicos e especificação do número de validações Cruzadas
RANDOM_STATE = 0
cv = 10

#Carregando o dataset utilizando o pandas
dataset = pd.read_csv("dataset_problema2.csv")

#Remoção de features desnecessárias pra o treinarmento e mantendo apenas o necessário para a predição
X = dataset.drop(["id", "Severidade"], axis=1)
y = dataset["Severidade"]

#Separação dos conjuntos de treino e teste (80% para treino e 20% para teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = RANDOM_STATE
)

#Função utilizada para a escolha e otimizacao dos hiperparâmetros
def train_model(parameters):
    n_estimators     = int(parameters[0])
    learning_rate    = float(parameters[1])
    max_depth        = int(parameters[2])
    subsample        = float(parameters[3])
    min_samples_leaf = int(parameters[4])
    max_features     = parameters[5]
    model = Pipeline(steps = [
        ("gb", GradientBoostingRegressor(
            n_estimators = n_estimators,
            learning_rate = learning_rate,
            max_depth = max_depth,
            subsample = subsample,
            min_samples_leaf = min_samples_leaf,
            max_features = max_features,
            random_state = RANDOM_STATE,
            loss = "squared_error"
        ))
    ])
    # Validação cruzada (neg_root_mean_squared_error → invertido para RMSE positivo)
    scores = cross_val_score(
        model, X_train, y_train,
        scoring = "neg_root_mean_squared_error",
        cv = cv,
        n_jobs = -1
    )
    rmse_mean = -scores.mean()
    return rmse_mean 

#Definição dos parametros a serem considerados pela funcao de otimizacao
parameters = [
    (150, 1200),            # n_estimators
    (1e-3, 0.2),            # learning_rate (uniform, NÃO log-uniform)
    (2, 6),                 # max_depth
    (0.6, 1.0),             # subsample
    (1, 80),                # min_samples_leaf
    ["sqrt", "log2", None], # max_features
]

#Realiza a busca pelos melhores hiperparametros utlizando a funcao train_model
otimos = gp_minimize(train_model, parameters, random_state = RANDOM_STATE, n_calls = 50, n_random_starts = 10)

#Valida uma referencia para os parametros ótimos
bestParams = otimos.x

#Cria umaa pipeline para configurar o modelo GradientBoostingRegressor com os melhores parâmetros
best_gradient_br = Pipeline(steps = [
    ("gb", GradientBoostingRegressor (
        n_estimators = int(bestParams[0]),
        learning_rate = float(bestParams[1]),
        max_depth = int(bestParams[2]),
        subsample = float(bestParams[3]),
        min_samples_leaf = int(bestParams[4]),
        max_features = bestParams[5],
        random_state = RANDOM_STATE,
        loss = "squared_error"
    ))
])

#Treinamento do modelo de GradienteBoostingRegressor com os melhores parâmetros encontrados
best_gradient_br.fit(X_train, y_train)
y_pred = best_gradient_br.predict(X_test)

#Cálculo e impressão das métricas
r2  = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae  = mean_absolute_error(y_test, y_pred)
print(r2)
print(rmse)
print(mae)


0.8979221223604349
7.186047102591083
6.1089153438473565
